In [1]:
# first change the directory over to ssbd release
import os 
import xml.etree.ElementTree as ET

# The first step is preprocessing. We use the [SSBD Dataset](https://rolandgoecke.net/research/datasets/ssbd/) in order to get the data for hand flapping and spinning. 

## The dataset does contain 75 URLs (although we'll ignore all headbanging videos) to youtube videos. All the data is nicely stored in XML files that I will read to get the youtube videos and also the time stamps of when the beheavior (hand flapping or spinning) occurs. Then I will use pytube to download the youtube videos to .mp4 and moviepy to cut the .mp4 videos into just the areas of interest. 

In [2]:
# next step would be to get the hand flapping and spinning data 

tree = ET.parse("ssbd-release/Annotations/v_ArmFlapping_07.xml")
root = tree.getroot()
for child in root:
    # for each child in the root 
    if child.tag == "url":
        print(child.text)
    if child.tag == "behaviours":
        for behavior in child: # go through each reported behavior 
            for tag in behavior: # tag is just the attribute of the behavior 
                if tag.tag == "time":
                    print(tag.text)
                if tag.tag == "intensity":
                    print(tag.text)
                if tag.tag == "category":
                    print(tag.text)

http://www.youtube.com/watch?v=T9rbit_oiJA
0004:0010
armflapping
high
0015:0020
armflapping
high


In [4]:
def convert_to_second(time : str) -> int:
    # this will take in a time like "0125" or 1:25 and make it 85 (60 + 25)
    overall_seconds = 0 
    for i, time_char in enumerate(reversed(time)):
        if i == 0:
            overall_seconds += int(time_char)
        if i == 1:
            overall_seconds += int(time_char) * 10
        if i == 2:
            overall_seconds += int(time_char) * 60 
        if i == 3:
            overall_seconds += int(time_char) * 600 
    return overall_seconds 

assert convert_to_second('2345') == 23 * 60 + 45  

In [5]:
from collections import defaultdict
URLS_TO_DOWNLOAD = set() # contains all youtube videos to download 
links_to_times = {} # data will be stored here like {link : {'category' : [(start, end)], 'another cat' : [(start, end)]}

for i, file_name in enumerate(os.listdir('ssbd-release/Annotations/')):
    # parse this file 
    tree = ET.parse('ssbd-release/Annotations/' + file_name)
    root = tree.getroot() 
    
    # everything we need to store
    URL = ""
    
    for child in root:
        
        if child.tag == "url":
            URL = child.text # store URL
            URLS_TO_DOWNLOAD.add(URL)
            links_to_times[URL] = defaultdict(list) 
        
        if child.tag == "behaviours": # this child has the list of behaviors 
            for reported_behavior in child: 
                for info in reported_behavior:
                    # gather the start time, end time, and category for this youtube link 
                    if info.tag == "time":
                        # the time will be start:end 
                        times = str(info.text) # contains the string 
                        if times.count(":"): 
                            divider_index = times.index(":")
                        elif times.count("-"):
                            divider_index = times.index("-")
                        else:
                            break # invalid then 
                        START_TIME, END_TIME = convert_to_second(times[:divider_index]), convert_to_second(times[divider_index + 1:])
                        
                    if info.tag == "category":
                        # this is the label 
                        LABEL = info.text 
                
                # create an entry for this reported behavior 
                links_to_times[URL][LABEL].append((START_TIME, END_TIME))

In [6]:
# pickle this just incase 
import pickle 
with open("links_to_times.pkl", 'wb') as f:
    pickle.dump(links_to_times, f)

In [ ]:
from moviepy.video.io.VideoFileClip import VideoFileClip

input_video_path = 'myPath/vid1.mp4'
output_video_path = 'myPath/output/vid1.mp4'

with VideoFileClip(input_video_path) as video:
    new = video.subclip(t1, t2)
    new.write_videofile(output_video_path, audio_codec='aac')

In [13]:
import cv2
import numpy as np
from moviepy.video.io.VideoFileClip import VideoFileClip
from pytube import YouTube # used to download the videos
FPS = 30 
for i, (url, category_times) in enumerate(links_to_times.items()): 
    
    # download the video 
    y = YouTube(url)
    try: 
        y.streams.get_highest_resolution().download() 
    except Exception as e:
        print(f"annoying url: {url}")
        continue 
    for category, times in category_times.items(): 
        if category == "headbanging":
            print("this is headbanging, so skipping")
            continue 
        folder_path = "behavior_data/" + category + "/"
        
        for start_time, end_time in times:
            try:
                input_file = y.streams.get_highest_resolution().default_filename
                output_file = folder_path + f"{i}.mp4"
                with VideoFileClip(input_file) as video:
                    new = video.subclip(start_time, end_time)
                    new.write_videofile(output_file, audio_codec='aac')
            except Exception as e:
                print(f"failed on {i}")
    os.remove(y.streams.get_highest_resolution().default_filename)

Moviepy - Building video behavior_data/spinning/0.mp4.
MoviePy - Writing audio in 0TEMP_MPY_wvf_snd.mp4
MoviePy - Done.
Moviepy - Writing video behavior_data/spinning/0.mp4



Moviepy - Done !
Moviepy - video ready behavior_data/spinning/0.mp4
this is headbanging, so skipping


t:   0%|          | 0/168 [00:00<?, ?it/s, now=None]                

Moviepy - Building video behavior_data/armflapping/2.mp4.
MoviePy - Writing audio in 2TEMP_MPY_wvf_snd.mp4
MoviePy - Done.
Moviepy - Writing video behavior_data/armflapping/2.mp4



Moviepy - Done !
Moviepy - video ready behavior_data/armflapping/2.mp4


...

Moviepy - Done !
Moviepy - video ready behavior_data/armflapping/71.mp4
Moviepy - Building video behavior_data/armflapping/71.mp4.
MoviePy - Writing audio in 71TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video behavior_data/armflapping/71.mp4



chunk:   0%|          | 0/111 [00:00<?, ?it/s, now=None]

Moviepy - Done !
Moviepy - video ready behavior_data/armflapping/71.mp4
Moviepy - Building video behavior_data/armflapping/71.mp4.
MoviePy - Writing audio in 71TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video behavior_data/armflapping/71.mp4

Moviepy - Done !
Moviepy - video ready behavior_data/armflapping/71.mp4
annoying url: http://www.youtube.com/watch?v=Pa-pdBF4FFA
this is headbanging, so skipping


# Directory Structure  

## We have one folder inside of this AnishMachineLearning folder called "behavior_data" that has the "armflapping" and "spinning" folders. There all of the sliced .mp4 files with the behavior of interest are located. 

### Note that not all of the 75 youtube videos from SSBD are there in the armflapping & spinning folder there because some have gone and some are for headbanging. 

In [ ]:
# for spinning it is just the hand positions that matter right?

